
# 📢 Code Showcase: How to Use the Latest Version of the Summarizer

In this sharing on GitHub, we will demonstrate how to use the latest version of our summarizer. This new version of the summarizer employs the output of DeepSeek-R1-671B as the target for distillation. Specifically, we use the disease entries from OMIM or Orphanet, along with their HPO annotation definitions, and input them into DeepSeek-R1-671B to generate synthetic simulated cases. For the detailed process, please refer to our manuscript.

Next, we will showcase two practical functions of the summarizer:
1. **Symptom Summary Generation**: Input the patient's HPO terms, and the summarizer will quickly generate a summary report of the patient's symptoms.
2. **Structured Clinical Report Generation**: Demonstrate how the summarizer receives the results from the ranker and the recommender, and then generates a structured clinical report.

The creator of this notebook: Baole Wen (2025.03.29)

In [ ]:
import re
import torch
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer

from pyhpo import Ontology

ontology = Ontology(data_folder='../data/hpo-2025-05-06')

In [2]:
def Get_Definition(hpo_list):
    definition_list = []
    for t in hpo_list:
        definition = Ontology.get_hpo_object(t).definition
        match = re.search(r'"(.*?)"', definition)
        if match:
            definition_list.append(match.group(1))
    return ' '.join(definition_list)

# HPO Terms Detected in a Patient

1. **HP:0000006** - Autosomal dominant inheritance  
2. **HP:0003593** - Infantile onset  
3. **HP:0025104** - Capillary malformation  
4. **HP:0001009** - Telangiectasia  
5. **HP:0003829** - Typified by incomplete penetrance  
6. **HP:0030713** - Vein of Galen aneurysmal malformation  


In [3]:
Patient_hps = (["HP:0000006", "HP:0003593", "HP:0025104", "HP:0001009", "HP:0003829", "HP:0030713"])

In [4]:
Input_text = Get_Definition(Patient_hps)

In [ ]:
model_path = '../data/model/Bio-Medical-3B-CoT-Finetuned'

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [ ]:
summaries = []

max_input_tokens = 2048

hpo_def = Input_text

prompt = f"""
I will provide you with the definitions of some HPO (Human Phenotype Ontology) terms exhibited by a patient. Based on these definitions, please generate a concise, clinically focused summary of the patient's symptoms in one paragraph, approximately 100-300 words in length. Ensure the summary is highly readable, with smooth transitions between ideas, logical coherence, and accurate representation of the clinical features. Emphasize clarity, fluency, and clinical relevance to create a realistic and precise description of the patient's presentation.\nText:\n{hpo_def}
"""
  
tokenized_text = tokenizer(prompt, return_tensors="pt").input_ids[0]
truncated_tokenized_text = tokenized_text[:max_input_tokens]
    
truncated_text = tokenizer.decode(truncated_tokenized_text)  + '<think>:\n'

summarizer = pipeline(
    "text-generation",  
    model=model,  
    tokenizer=tokenizer
)
response = summarizer(
    truncated_text,
    max_new_tokens= max_input_tokens + 1024,
    top_p=0.95,
    top_k=50,
    do_sample=True
)

summary = response[0]['generated_text'].split('<think>:')
summaries.append(summary)
torch.cuda.empty_cache()


## Input prompt

In [ ]:
print(summaries[0][0])

## Output

In [ ]:
print(summaries[0][1])

### Case Demonstration
In the following case, we first used the prompt text generated in the example tutorials of the **Rank** and **Recommender**. This prompt text integrates the patient's input symptom information as well as the results from the Rank and Recommender. 

In [ ]:
with open('../data/case_report_prompt.txt', 'r', encoding='utf-8') as file:
    content = file.read()
    print(content)


In [ ]:
summaries = []

max_input_tokens = 4096

hpo_def = Input_text

prompt = content
  
tokenized_text = tokenizer(prompt, return_tensors="pt").input_ids[0]
truncated_tokenized_text = tokenized_text[:max_input_tokens]
    
truncated_text = tokenizer.decode(truncated_tokenized_text)  + '<think>:\n'

summarizer = pipeline(
    "text-generation",  
    model=model,  
    tokenizer=tokenizer
)
response = summarizer(
    truncated_text,
    max_new_tokens= max_input_tokens + 4096,
    top_p=0.95,
    top_k=50,
    do_sample=True
)

summary = response[0]['generated_text'].split('<think>:')
summaries.append(summary)
torch.cuda.empty_cache()


# Input

In [ ]:
print(summaries[0][0])

# Output

In [ ]:
print(summaries[0][1])

# Chain of thought

In [ ]:
print(summaries[0][1].split('</think>')[0])

# Case Report

In [ ]:
print(summaries[0][1].split('</think>')[1])